In [1]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import joblib
import numpy as np
import json
import os

df = pd.read_csv("../data/processed/transformed_data.csv")

In [3]:
!pip install xgboost

In [5]:
#input features and target features
input_features = [
    "championId", "matchupChampion", "individualPosition", 
    "kills", "deaths", "assists", 
    "goldEarned", "totalDamageDealt", 
    "totalDamageTaken", "totalHeal", "win"
]
target_features = [
    "Boots_id", "Legendary_1_id", "Legendary_2_id",
    "Keystone", "PrimarySlot1", "PrimarySlot2",
    "PrimarySlot3", "SecondarySlot1", "SecondarySlot2"
]

# onehot encoding individualPosition
one_hot_encoder = OneHotEncoder(drop='first', handle_unknown='ignore')

# scaling numerical features
scaler = StandardScaler()

# preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ("position", one_hot_encoder, ["individualPosition"]),
        ("scaling", scaler, ["kills", "deaths", "assists", "goldEarned", "totalDamageDealt", "totalDamageTaken", "totalHeal"])
    ],
    remainder='passthrough'  #
)

# create pipeline for data processing
pipeline = Pipeline([
    ("preprocessor", preprocessor)
])

In [7]:
# fit and transform the input features
X = df[input_features]
X_processed = pipeline.fit_transform(X)

# target features (outputs)
y = df[target_features]

In [9]:
from sklearn.preprocessing import LabelEncoder

# Initialize encoders for each target column
label_encoders = {}
encoded_y = pd.DataFrame()

# Loop through each target column and encode it
for col in target_features:
    le = LabelEncoder()
    encoded_y[col] = le.fit_transform(y[col])
    label_encoders[col] = le 

label_encoders_path = "../models/label_encoders.pkl"
joblib.dump(label_encoders, label_encoders_path)

['../models/label_encoders.pkl']

In [13]:
from xgboost import XGBClassifier

# initialize the base classifier
base_classifier = XGBClassifier(n_estimators=100, random_state=42, use_label_encoder=False, eval_metric='logloss') # change n_estimators back to 100 for optimal results
multi_target_classifier = MultiOutputClassifier(base_classifier)

# fitting the classifier
multi_target_classifier.fit(X_processed, encoded_y)

C:\Users\Natan\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:18:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Natan\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:18:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Natan\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:18:55] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Natan\anaconda3\Lib\site-packages\

MultiOutputClassifier(estimator=XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              device=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='logloss',
                                              feature_types=None, gamma=None,
                                              grow_policy=None,
                                              importance_type=None,
                                              interaction_constraints=None,
                                              learning_rate=None, max_bin=None,
                                              max_cat_threshold=None,
                                              max_cat_to_onehot=None,
                                              max_delta_step=None,
                                              max_depth=None, max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              multi_strategy=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              random_state=42, ...))

In [71]:
# Predicting
predicted_encoded_output = multi_target_classifier.predict(X_processed)

# Convert the prediction to a DataFrame
predicted_encoded_df = pd.DataFrame(predicted_encoded_output, columns=target_features)

# Decode the predictions using the stored LabelEncoders
predicted_decoded_df = pd.DataFrame()
for col in target_features:
    predicted_decoded_df[col] = label_encoders[col].inverse_transform(predicted_encoded_df[col])

In [53]:
# loading champion, item, and rune datasets
with open("../data/raw/champion_data/champions.json", "r") as f:
    champion_data = json.load(f)["data"]

with open("../data/raw/item_data/items.json", "r") as f:
    item_data = json.load(f)["data"]

with open("../data/raw/runes_data/runes.json", "r") as f:
    rune_data = json.load(f)

# creating lookup dictionaries
champion_name_to_id = {v["name"].lower(): int(v["key"]) for k, v in champion_data.items()}
champion_id_to_name = {int(v["key"]): v["name"] for k, v in champion_data.items()}

item_id_to_name = {int(k): v["name"] for k, v in item_data.items()}

# create lookup dictionaries for rune names and rune trees
rune_id_to_name = {}
rune_id_to_tree = {}
rune_id_to_row = {}
rune_trees = []

for style in rune_data:
    # add the main style name
    rune_id_to_name[style["id"]] = style["name"]
    rune_trees.append(style["id"])  # Store available rune trees

    # add individual runes within each style and map each rune to its tree and row
    for row_idx, slot in enumerate(style["slots"]):
        for rune in slot["runes"]:
            rune_id_to_name[rune["id"]] = rune["name"]
            rune_id_to_tree[rune["id"]] = style["id"]
            rune_id_to_row[rune["id"]] = row_idx

In [65]:
# baseline model creation

def predict_optimal_build(champion_name, matchup_champion_name, df, pipeline, model, label_encoders):
    """
    Predict the optimal item build and runes for the given champion and matchup champion.
    
    Parameters:
    - champion_name: str, champion name of the player.
    - matchup_champion_name: str, champion name of the opponent.
    - df: DataFrame, original DataFrame with historical data.
    - pipeline: preprocessing pipeline used for transforming the features.
    - model: trained MultiOutputClassifier model.
    - label_encoders: dict, dictionary of LabelEncoders for each target feature.

    Returns:
    - DataFrame, containing the predicted items and runes.
    """
    # Convert champion names to IDs
    champion_id = champion_name_to_id.get(champion_name.lower())
    matchup_champion_id = champion_name_to_id.get(matchup_champion_name.lower())

    if champion_id is None or matchup_champion_id is None:
        raise ValueError(f"Champion name(s) provided are not valid: {champion_name}, {matchup_champion_name}")

    # Create a new input DataFrame with average values for other features
    input_data = df[(df['championId'] == champion_id) & (df['matchupChampion'] == matchup_champion_id)].mean().to_dict()

    # Check if input_data has NaN values and provide default values if needed
    if any(pd.isna(value) for value in input_data.values()):
        raise ValueError(f"No data available for the matchup: {champion_name} vs {matchup_champion_name}")

    # Override champion-specific fields
    input_data['championId'] = champion_id
    input_data['matchupChampion'] = matchup_champion_id

    # Create a DataFrame for input
    input_features = df.columns.difference(target_features)
    input_df = pd.DataFrame([input_data], columns=input_features)

    # Preprocess the input features using the pipeline
    input_processed = pipeline.transform(input_df)

    # Predict the output
    predicted_output = model.predict(input_processed)

    # Convert the prediction to a DataFrame for easier handling
    predicted_encoded_df = pd.DataFrame(predicted_output, columns=target_features)

    # Decode the predictions using the stored LabelEncoders
    predicted_decoded_df = pd.DataFrame()
    for col in target_features:
        predicted_decoded_df[col] = label_encoders[col].inverse_transform(predicted_encoded_df[col])

    # Validate rune selections - ensure they come from the same tree
    primary_tree = rune_id_to_tree.get(predicted_decoded_df['Keystone'][0], None)
    for col in ['PrimarySlot1', 'PrimarySlot2', 'PrimarySlot3']:
        if rune_id_to_tree.get(predicted_decoded_df[col][0]) != primary_tree:
            # Replace with the most frequent rune from the primary tree if it doesn't match
            valid_runes = df[(df['Keystone'] == predicted_decoded_df['Keystone'][0])][col].value_counts().index.tolist()
            if valid_runes:
                predicted_decoded_df[col] = valid_runes[0]

    # Validate that secondary runes come from the correct tree
    secondary_tree = rune_id_to_tree.get(predicted_decoded_df['SecondarySlot1'][0], None)
    if secondary_tree == primary_tree:
        # Ensure secondary tree is different from the primary tree
        secondary_tree_options = [t for t in rune_trees if t != primary_tree]
        secondary_tree = secondary_tree_options[0]  # Replace with the first different tree found

    for col in ['SecondarySlot1', 'SecondarySlot2']:
        if rune_id_to_tree.get(predicted_decoded_df[col][0]) != secondary_tree:
            if col == 'SecondarySlot2':
                # If `SecondarySlot2` doesn't match, find the most frequent rune that pairs well historically
                valid_runes = df[(df['championId'] == champion_id) & 
                                 (df['matchupChampion'] == matchup_champion_id) & 
                                 (df['SecondarySlot1'] == predicted_decoded_df['SecondarySlot1'][0])][col].value_counts().index.tolist()
                if valid_runes:
                    predicted_decoded_df[col] = valid_runes[0]

    # Convert IDs to item and rune names for user-friendly output
    for col in ['Boots_id', 'Legendary_1_id', 'Legendary_2_id']:
        predicted_decoded_df[col] = predicted_decoded_df[col].apply(lambda x: item_id_to_name.get(int(x), "Unknown Item"))

    for col in ['Keystone', 'PrimarySlot1', 'PrimarySlot2', 'PrimarySlot3', 'SecondarySlot1', 'SecondarySlot2']:
        predicted_decoded_df[col] = predicted_decoded_df[col].apply(lambda x: rune_id_to_name.get(int(x), "Unknown Rune"))

    return predicted_decoded_df

# Example Usage
champion_name = "Syndra"  
matchup_champion_name = "Azir"  

try:
    # Predict the optimal build
    predicted_build = predict_optimal_build(champion_name, matchup_champion_name, df, pipeline, multi_target_classifier, label_encoders)

    # Print results
    print(f"Suggested Items and Runes for the given matchup: {champion_name} vs {matchup_champion_name}")
    print(predicted_build)
except ValueError as e:
    print(f"Error: {e}")

Suggested Items and Runes for the given matchup: Syndra vs Azir
           Boots_id     Legendary_1_id Legendary_2_id      Keystone  \
0  Sorcerer's Shoes  Luden's Companion     Stormsurge  First Strike   

       PrimarySlot1      PrimarySlot2    PrimarySlot3 SecondarySlot1  \
0  Magical Footwear  Biscuit Delivery  Cosmic Insight  Manaflow Band   

  SecondarySlot2  
0  Transcendence  


In [63]:
# saving the model to a file
joblib.dump(multi_target_classifier, "../models/best_recommendation_model.pkl", protocol=4)

['../models/best_recommendation_model.pkl']